### Import Dependencies

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import torch
from torchvision import datasets, transforms, models  # datsets  , transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import Dataset

<b> Dataset Link (Plant Vliiage Dataset ):</b><br> <a href='https://data.mendeley.com/datasets/tywbtsjrjv/1'> https://data.mendeley.com/datasets/tywbtsjrjv/1 </a> 

In [5]:
transform = transforms.Compose(
    [transforms.Resize(255), transforms.CenterCrop(224), transforms.ToTensor()]
)

In [6]:
!pip install rarfile  # Install the rarfile package if it's not already installed

import os
from rarfile import RarFile
from torchvision import transforms, datasets
# Define the path to the RAR file
rar_path = '/content/drive/MyDrive/Dataset.rar'

# Extract the RAR file to a temporary directory
with RarFile(rar_path, 'r') as rar:
    rar.extractall('/tmp/dataset')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
# Create the PyTorch dataset from the extracted images
dataset = datasets.ImageFolder('/tmp/dataset', transform=transform)
dataset

Dataset ImageFolder
    Number of datapoints: 55448
    Root location: /tmp/dataset
    StandardTransform
Transform: Compose(
               Resize(size=255, interpolation=bilinear, max_size=None, antialias=warn)
               CenterCrop(size=(224, 224))
               ToTensor()
           )

In [8]:
indices = list(range(len(dataset)))


In [9]:
split = int(np.floor(0.85 * len(dataset)))  # train_size

In [10]:
validation = int(np.floor(0.8 * split))  # validation

In [11]:
print(0, validation, split, len(dataset))

0 37704 47130 55448


In [12]:
print(f"length of train size :{validation}")
print(f"length of validation size :{split - validation}")
print(f"length of test size :{len(dataset)-validation}")

length of train size :37704
length of validation size :9426
length of test size :17744


In [13]:
np.random.shuffle(indices)

### Split into Train and Test

In [14]:
train_indices, validation_indices, test_indices = (
    indices[:validation],
    indices[validation:split],
    indices[split:],
)

In [15]:
train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(validation_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [16]:
# targets_size = len(dataset.class_to_idx)
targets_size = 39
print(targets_size)

39


### Model

<b>Convolution Aithmetic Equation : </b>(W - F + 2P) / S + 1 <br>
W = Input Size<br>
F = Filter Size<br>
P = Padding Size<br>
S = Stride <br>

### Transfer Learning

In [17]:
model = models.vgg16(pretrained=True)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 81.1MB/s]


In [18]:
for params in model.parameters():
    params.requires_grad = False

In [19]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [20]:
n_features = model.classifier[0].in_features
n_features

25088

In [21]:
model.classifier = nn.Sequential(
    nn.Linear(n_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(1024, targets_size),
)

In [22]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Original Modeling

In [23]:
# class CNN(nn.Module):
#     def __init__(self, K):
#         super(CNN, self).__init__()
#         self.conv_layers = nn.Sequential(
#             # conv1
#             nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(32),
#             nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(32),
#             nn.MaxPool2d(2),
#             # conv2
#             nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(64),
#             nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(64),
#             nn.MaxPool2d(2),
#             # conv3
#             nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(128),
#             nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(128),
#             nn.MaxPool2d(2),
#             # conv4
#             nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(256),
#             nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(256),
#             nn.MaxPool2d(2),
#         )

#         self.dense_layers = nn.Sequential(
#             nn.Dropout(0.4),
#             nn.Linear(50176, 1024),
#             nn.ReLU(),
#             nn.Dropout(0.4),
#             nn.Linear(1024, K),
#         )

#     def forward(self, X):
#         out = self.conv_layers(X)

#         # Flatten
#         out = out.view(-1, 50176)

#         # Fully connected
#         out = self.dense_layers(out)

#         return out

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [25]:
# model = model(targets_size)

In [26]:
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [27]:
# from torchsummary import summary

# summary(model, (3, 224, 224))

In [28]:
from torchsummary import summary

summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [29]:
criterion = nn.CrossEntropyLoss()  # this include softmax + cross entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Batch Gradient Descent

In [30]:
# def batch_gd(model, criterion, train_loader, validation_loader, epochs):
#     train_losses = np.zeros(epochs)
#     validation_losses = np.zeros(epochs)

#     for e in range(epochs):
#         t0 = datetime.now()
#         train_loss = []
#         for inputs, targets in train_loader:
#             inputs, targets = inputs.to(device), targets.to(device)

#             optimizer.zero_grad()

#             output = model(inputs)

#             loss = criterion(output, targets)

#             train_loss.append(loss.item())  # torch to numpy world

#             loss.backward()
#             optimizer.step()

#         train_loss = np.mean(train_loss)

#         validation_loss = []

#         for inputs, targets in validation_loader:

#             inputs, targets = inputs.to(device), targets.to(device)

#             output = model(inputs)

#             loss = criterion(output, targets)

#             validation_loss.append(loss.item())  # torch to numpy world

#         validation_loss = np.mean(validation_loss)

#         train_losses[e] = train_loss
#         validation_losses[e] = validation_loss

#         dt = datetime.now() - t0

#         print(
#             f"Epoch : {e+1}/{epochs} Train_loss:{train_loss:.3f} Test_loss:{validation_loss:.3f} Duration:{dt}"
#         )

#     return train_losses, validation_losses
def batch_gd(model, criterion, train_loader, validation_loader, epochs, optimizer):
    train_losses = np.zeros(epochs)
    validation_losses = np.zeros(epochs)

    for e in range(epochs):
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            output = model(inputs)

            loss = criterion(output, targets)

            train_loss.append(loss.item())  # torch to numpy world

            loss.backward()
            optimizer.step()

        train_loss = np.mean(train_loss)

        validation_loss = []
        for inputs, targets in validation_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            output = model(inputs)
            loss = criterion(output, targets)
            validation_loss.append(loss.item())  # torch to numpy world

        validation_loss = np.mean(validation_loss)

        train_losses[e] = train_loss
        validation_losses[e] = validation_loss

        dt = datetime.now() - t0

        print(
            f"Epoch : {e+1}/{epochs} Train_loss:{train_loss:.3f} Validation_loss:{validation_loss:.3f} Duration:{dt}"
        )

    return train_losses, validation_losses


In [31]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=train_sampler
)
test_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=test_sampler
)
validation_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=validation_sampler
)

In [ ]:
train_losses, validation_losses = batch_gd(
    model, criterion, train_loader, validation_loader, 1, optimizer
)

### Save the Model

In [ ]:
torch.save(model.state_dict(), "plant_disease_model_1.pt")

### Load Model

In [ ]:
targets_size = 1
# model = CNN(targets_size)
num_features = model.classifier[-1].in_features
classifier = nn.Linear(num_features, targets_size)
model.classifier[-1] = classifier
model.load_state_dict(torch.load("plant_disease_model_1.pt"))
model.eval()

In [ ]:
# %matplotlib notebook

### Plot the loss

In [ ]:
plt.plot(train_losses, label="train_loss")
plt.plot(validation_losses, label="validation_loss")
plt.xlabel("No of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

### Accuracy

In [ ]:
def accuracy(loader):
    n_correct = 0
    n_total = 0
    
    
    # Move the model to the GPU
    model.to(device)

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)

        _, predictions = torch.max(outputs, 1)

        n_correct += (predictions == targets).sum().item()
        n_total += targets.shape[0]

    acc = n_correct / n_total
    return acc

In [ ]:
train_acc = accuracy(train_loader)
test_acc = accuracy(test_loader)
validation_acc = accuracy(validation_loader)

In [ ]:
print(
    f"Train Accuracy : {train_acc}\nTest Accuracy : {test_acc}\nValidation Accuracy : {validation_acc}"
)

### Single Image Prediction

In [ ]:
transform_index_to_disease = dataset.class_to_idx

<IPython.core.display.Javascript object>

In [ ]:
transform_index_to_disease = dict(
    [(value, key) for key, value in transform_index_to_disease.items()]
)  # reverse the index

<IPython.core.display.Javascript object>

In [ ]:
data = pd.read_csv("disease_info.csv", encoding="cp1252")

<IPython.core.display.Javascript object>

In [ ]:
from PIL import Image
import torchvision.transforms.functional as TF

<IPython.core.display.Javascript object>

In [ ]:
def single_prediction(image_path):
    image = Image.open(image_path)
    image = image.resize((224, 224))
    input_data = TF.to_tensor(image)
    input_data = input_data.view((-1, 3, 224, 224))
    output = model(input_data)
    output = output.detach().numpy()
    index = np.argmax(output)
    print("Original : ", image_path[12:-4])
    pred_csv = data["disease_name"][index]
    print(pred_csv)

<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/Apple_ceder_apple_rust.JPG")

Original :  Apple_ceder_apple_rust
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

### Wrong Prediction

In [ ]:
single_prediction("test_images/Apple_scab.JPG")

Original :  Apple_scab
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/Grape_esca.JPG")

Original :  Grape_esca
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/apple_black_rot.JPG")

Original :  apple_black_rot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/apple_healthy.JPG")

Original :  apple_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/background_without_leaves.jpg")

Original :  background_without_leaves
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/blueberry_healthy.JPG")

Original :  blueberry_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/cherry_healthy.JPG")

Original :  cherry_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/cherry_powdery_mildew.JPG")

Original :  cherry_powdery_mildew
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/corn_cercospora_leaf.JPG")

Original :  corn_cercospora_leaf
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/corn_common_rust.JPG")

Original :  corn_common_rust
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/corn_healthy.jpg")

Original :  corn_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/corn_northen_leaf_blight.JPG")

Original :  corn_northen_leaf_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/grape_black_rot.JPG")

Original :  grape_black_rot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/grape_healthy.JPG")

Original :  grape_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/grape_leaf_blight.JPG")

Original :  grape_leaf_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/orange_haunglongbing.JPG")

Original :  orange_haunglongbing
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/peach_bacterial_spot.JPG")

Original :  peach_bacterial_spot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/peach_healthy.JPG")

Original :  peach_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/pepper_bacterial_spot.JPG")

Original :  pepper_bacterial_spot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/pepper_bell_healthy.JPG")

Original :  pepper_bell_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/potato_early_blight.JPG")

Original :  potato_early_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/potato_healthy.JPG")

Original :  potato_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/potato_late_blight.JPG")

Original :  potato_late_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/raspberry_healthy.JPG")

Original :  raspberry_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/soyaben healthy.JPG")

Original :  soyaben healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/potato_late_blight.JPG")

Original :  potato_late_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/squash_powdery_mildew.JPG")

Original :  squash_powdery_mildew
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/starwberry_healthy.JPG")

Original :  starwberry_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/starwberry_leaf_scorch.JPG")

Original :  starwberry_leaf_scorch
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/tomato_bacterial_spot.JPG")

Original :  tomato_bacterial_spot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/tomato_early_blight.JPG")

Original :  tomato_early_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/tomato_healthy.JPG")

Original :  tomato_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/tomato_late_blight.JPG")

Original :  tomato_late_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/tomato_leaf_mold.JPG")

Original :  tomato_leaf_mold
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/tomato_mosaic_virus.JPG")

Original :  tomato_mosaic_virus
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/tomato_septoria_leaf_spot.JPG")

Original :  tomato_septoria_leaf_spot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/tomato_spider_mites_two_spotted_spider_mites.JPG")

Original :  tomato_spider_mites_two_spotted_spider_mites
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/tomato_target_spot.JPG")

Original :  tomato_target_spot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [ ]:
single_prediction("test_images/tomato_yellow_leaf_curl_virus.JPG")

Original :  tomato_yellow_leaf_curl_virus
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>